In [ ]:
from importlib import reload

import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp

import L96
import relax_punch

reload(L96)
reload(relax_punch)

ndarray = np.ndarray

In [ ]:
def evolve(
    system: L96.System,
    tlim: tuple[float],
    U0: ndarray,
    V0: ndarray,
    U0_sim: ndarray,
    V0_sim: ndarray,
):
    """Evolve the true and simulated system from tlim[0] to tlim[1].

    Parameters
    ----------
    system
        Instance of `L96.System`
    tlim
        (start time, stop time)
    U0
        The initial state of the true large-scale system
        shape (I,)
    V0
        The initial states of the true small-scale systems
        shape (I, J)
    U0_sim, V0_sim
        The initial states of the simulated large- and small-scale systems
        shapes same as those for U0 and V0
    """

    state0 = L96.together(U0, V0)
    state0_sim = L96.together(U0_sim, V0_sim)

    # Evolve true and simulated systems
    sol = solve_ivp(
        system.ode_true,
        tlim,
        state0,
        dense_output=True,
    )

    sim = solve_ivp(
        system.ode_sim,
        tlim,
        state0_sim,
        args=(sol.sol,),
        dense_output=True,
    )

    return sol, sim

In [ ]:
# Dimensions
I, J = 10, 3
J_sim = J - 2

# System evolution parameters
tlim = t0, tf = 0, 2

Δt = 5
γ1, γ2 = 1, 1
c1, c2 = 1, 1
ds2 = np.full(J, 1)
F = -1
μ = 10

rp = relax_punch.RelaxPunch(I, J, J_sim, Δt, γ1, γ2, ds2, c1, c2, F, μ)

# Initial true state
U0 = np.full(I, 1)
V0 = np.full((I, J), 2)

# Initial simulation state
U0_sim = U0 + 1
V0_sim = np.full((I, J_sim), 2)

sol, sim = evolve(rp.system, tlim, U0, V0, U0_sim, V0_sim)

tn = 100
tls = np.linspace(*tlim, tn)

# Unpack true and simulated states
states = sol.sol(tls)
Us, Vs = zip(*(L96.apart(state, I, J) for state in states.T))
Us, Vs = np.stack(Us), np.stack(Vs)

states_sim = sim.sol(tls)
Us_sim, Vs_sim = zip(*(L96.apart(state, I, J_sim) for state in states_sim.T))
Us_sim, Vs_sim = np.stack(Us_sim), np.stack(Vs_sim)

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(tls, Us.T[0], label="true", color="blue")
ax.plot(tls, Us_sim.T[0], label="sim", color="red", linestyle="--")

ax.legend()
plt.show()

In [ ]:
# TODO: Perform gradient descent update, then simulate again with new
# parameters.

# Learning Parameters
g = np.array([1e-2,0.5]) # True
paramErr = np.abs(np.random.randn(2)*1e-1)
c_init = 2*g  # Approximate
# Fixed Parameters
d = np.array([0.2,0.5,1,2,5]) #dissipation coefficients for the v_{k,j}
F=8. #forcing constant
mu=50.